In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df2017 = pd.read_table("PartD_Prescriber_PUF_NPI_17.txt")

c:\python\python35\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df2017.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip5,nppes_provider_zip4,nppes_provider_state,nppes_provider_country,specialty_description,description_flag,medicare_prvdr_enroll_status,total_claim_count,total_30_day_fill_count,total_drug_cost,total_day_supply,bene_count,ge65_suppress_flag,total_claim_count_ge65,total_30_day_fill_count_ge65,total_drug_cost_ge65,total_day_supply_ge65,bene_count_ge65_suppress_flag,bene_count_ge65,brand_suppress_flag,brand_claim_count,brand_drug_cost,generic_suppress_flag,generic_claim_count,generic_drug_cost,other_suppress_flag,other_claim_count,other_drug_cost,mapd_suppress_flag,mapd_claim_count,mapd_drug_cost,pdp_suppress_flag,pdp_claim_count,pdp_drug_cost,lis_suppress_flag,lis_claim_count,lis_drug_cost,nonlis_suppress_flag,nonlis_claim_count,nonlis_drug_cost,opioid_claim_count,opioid_drug_cost,opioid_day_supply,opioid_bene_count,opioid_prescriber_rate,la_opioid_claim_count,la_opioid_drug_cost,la_opioid_day_supply,la_opioid_bene_count,la_opioid_prescriber_rate,antibiotic_claim_count,antibiotic_drug_cost,antibiotic_bene_count,antipsych_ge65_suppress_flag,antipsych_claim_count_ge65,antipsych_drug_cost_ge65,antipsych_bene_ge65_suppress_flg,antipsych_bene_count_ge65,average_age_of_beneficiaries,beneficiary_age_less_65_count,beneficiary_age_65_74_count,beneficiary_age_75_84_count,beneficiary_age_greater_84_count,beneficiary_female_count,beneficiary_male_count,beneficiary_race_white_count,beneficiary_race_black_count,beneficiary_race_asian_pi_count,beneficiary_race_hispanic_count,beneficiary_race_nat_ind_count,beneficiary_race_other_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score
0,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,21502.0,1854.0,MD,US,Internal Medicine,S,E,677,695.5,32639.57,14788,234.0,NaN,516.0,532.533333,23174.78,11428.0,NaN,182.0,#,NaN,NaN,NaN,552.0,5941.53,*,NaN,NaN,NaN,150.0,3032.66,NaN,527.0,29606.91,NaN,305.0,17267.72,NaN,372.0,15371.85,25.0,197.47,134.0,24.0,3.69,NaN,NaN,NaN,NaN,NaN,125.0,2478.65,104.0,*,NaN,NaN,*,NaN,72.0,52.0,82.0,67.0,33.0,142.0,92.0,220.0,14.0,0.0,0.0,0.0,0.0,143.0,91.0,2.1685
1,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,43623.0,3536.0,OH,US,Anesthesiology,S,E,1946,2054.8,140189.01,58605,276.0,NaN,881.0,919.066667,50838.50,26089.0,NaN,160.0,NaN,235.0,106048.28,NaN,1711.0,34140.73,NaN,0.0,0.0,NaN,930.0,67081.22,NaN,1016.0,73107.79,NaN,1184.0,94361.07,NaN,762.0,45827.94,1040.0,50895.21,29238.0,200.0,53.44,200.0,37637.86,5853.0,49.0,19.23,0.0,0.00,0.0,NaN,0.0,0.0,NaN,0.0,65.0,116.0,99.0,45.0,16.0,184.0,92.0,195.0,58.0,NaN,NaN,0.0,NaN,143.0,133.0,1.8029
2,1003000167,ESCOBAR,JULIO,E,DDS,M,I,5 PINE CONE RD,NaN,DAYTON,89403.0,7482.0,NV,US,Dentist,S,N,55,55.0,302.01,430,33.0,#,NaN,NaN,NaN,NaN,#,NaN,NaN,0.0,0.00,#,NaN,NaN,*,NaN,NaN,NaN,16.0,58.19,NaN,39.0,243.82,*,NaN,NaN,#,NaN,NaN,14.0,61.95,35.0,11.0,25.45,0.0,0.00,0.0,0.0,0.00,28.0,134.92,22.0,NaN,0.0,0.0,NaN,0.0,72.0,NaN,NaN,NaN,NaN,16.0,17.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,1.0598
3,1003000175,REYES-VASQUEZ,BELINDA,NaN,D.D.S.,F,I,322 N AZUSA AVE STE 202,NaN,LA PUENTE,91744.0,4648.0,CA,US,Dentist,S,N,18,18.0,113.81,150,NaN,#,NaN,NaN,NaN,NaN,#,NaN,NaN,0.0,0.00,NaN,18.0,113.81,NaN,0.0,0.0,*,NaN,NaN,*,NaN,NaN,#,NaN,NaN,*,NaN,NaN,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,NaN,12.0,81.54,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1003000282,BLAKEMORE,ROSIE,K,FNP,F,I,TENNESSEE PRISON FOR WOMEN,3881 STEWARTS LANE,NASHVILLE,37243.0,1.0,TN,US,Nurse Practitioner,S,N,90,110.0,7561.21,2681,11.0,NaN,65.0,85.000000,3894.08,1931.0,*,NaN,NaN,28.0,6576.28,NaN,62.0,984.93,NaN,0.0,0.0,NaN,22.0,485.04,NaN,68.0,7076.17,NaN,73.0,3947.56,NaN,17.0,3613.65,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,NaN,0.0,0.00,0.0,NaN,0.0,0.0,NaN,0.0,

In [96]:
# Average number of beneficiaries per provider
average_bene = (df2017.bene_count.sum())/(df2017.bene_count.notna().sum())
print(average_bene)

158.3494585173676


In [83]:
# Length of the average prescription
# Check for zero:
if df2017.total_claim_count.isna().sum() == 0:
    df2017['average_length'] = df2017.total_day_supply / df2017.total_claim_count
print(df2017.average_length.median())

29.7125748502994


In [56]:
temp = df2017[df2017.brand_claim_count.notna()].groupby(['specialty_description'], as_index = False).agg(
        {'total_claim_count': ['sum'], 'brand_claim_count': ['sum']})


In [67]:
temp['ratio'] = temp.brand_claim_count['sum']/temp.total_claim_count['sum']

In [77]:
# STD of brand named ratio by specialty for specialties with over 1000 claims
print(temp[temp.total_claim_count['sum']>=1000].ratio.std())

0.09664998667912823


In [81]:
# Assuming we only care about regular opioid claims, not long-acting opioids
# Also assuming all states, the question was not specific to continental US
temp = df2017.groupby(['nppes_provider_state'], as_index = False).agg(
        {'opioid_bene_count': ['sum'], 'antibiotic_bene_count': ['sum']})


In [83]:
temp['ratio'] = temp.opioid_bene_count['sum']/temp.antibiotic_bene_count['sum']

In [89]:
max(temp.ratio)-min(temp.ratio)

0.6331097230267027

In [99]:
# What is the Pearson correlation coefficient between he fraction of claims for beneficiaries age 65 and older, 
# and the fraction of claims for beneficiaries with a low-income subsidy.
# Assuming that when the question states columns that are not suppressed, the data is not used if any of the columns are suppressed.
cols = ['beneficiary_age_65_74_count', 'beneficiary_age_75_84_count', 'beneficiary_age_greater_84_count','lis_claim_count']
temp = df2017.dropna(subset = cols)[cols]
temp['over_65'] = temp[cols[:-1]].sum(axis=1)

In [102]:
temp[['over_65','lis_claim_count']].corr()

,over_65,lis_claim_count
over_65,1.000000,0.323951
lis_claim_count,0.323951,1.000000


In [64]:
# Let's find which states have surprisingly high supply of opioids, conditioned on specialty.
temp = df2017[['nppes_provider_state','specialty_description', 'opioid_day_supply', 'opioid_claim_count', 'npi']]
temp['ratio'] = temp.opioid_day_supply / temp.opioid_claim_count

c:\python\python35\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
temp.head()

,nppes_provider_state,specialty_description,opioid_day_supply,opioid_claim_count,npi,ratio
0,MD,Internal Medicine,134.0,25.0,1003000126,5.360000
1,OH,Anesthesiology,29238.0,1040.0,1003000142,28.113462
2,NV,Dentist,35.0,14.0,1003000167,2.500000
3,CA,Dentist,0.0,0.0,1003000175,NaN
4,TN,Nurse Practitioner,0.0,0.0,1003000282,NaN


In [7]:
pair_count = temp.groupby(['nppes_provider_state', 'specialty_description'], as_index = False).agg(
        {'npi':['count']})

In [62]:
pair_set = set()
for index, row in pair_count[pair_count.npi['count'] >= 100].iterrows():
    pair_set.add((row.nppes_provider_state.item(), row.specialty_description.item()))

In [65]:
def pair_counts(row):
    state = row.nppes_provider_state
    job = row.specialty_description
    if (state, job) in pair_set:
        return 1
    else: 
        return 0
    

In [66]:
temp['counts'] = temp.apply(lambda row: pair_counts(row), axis = 1)

c:\python\python35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [73]:
average_ratio = temp[temp['counts'] == 1].ratio.mean()
print(average_ratio)

15.215713525165615


In [78]:
temp_state = temp[temp['counts'] == 1].groupby('specialty_description').agg({'ratio':'mean'})
temp_state['scaled_ratio'] = average_ratio/temp_state.ratio

In [81]:
max_ratio = max(temp_state.scaled_ratio)
print(max_ratio)

4.425436574456785

In [88]:
# Average cost per day of prescriptions in both 2016 and 2017
# We know the average length of a prescription already, now just divide the cost by the length in days
df2017['cost_per_day'] = df2017.total_drug_cost / df2017.average_length

In [91]:
average_cost_2017 = df2017['cost_per_day'].mean()
print(average_cost_2017)

3499.612480078


In [92]:
df2016 = pd.read_table("PartD_Prescriber_PUF_NPI_16.txt")

c:\python\python35\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [93]:
if df2016.total_claim_count.isna().sum() == 0:
    df2016['average_length'] = df2016.total_day_supply / df2016.total_claim_count
df2016['cost_per_day'] = df2016.total_drug_cost / df2016.average_length
average_cost_2016 = df2016['cost_per_day'].mean()
print(average_cost_2016)

In [95]:
inflation = average_cost_2017 / average_cost_2016
print(inflation)

1.0030290500719194


In [169]:
# Find the fraction of providers who left each specialty between 2016 and 2017.
temp2016 = df2016.groupby('specialty_description', as_index = False).agg({'npi':['count']})

In [170]:
temp2016.head()

,specialty_description,npi
,,count
0,Acupuncturist,86
1,Addiction Medicine,217
2,Advanced Practice Dental Therapist,2
3,Allergy/ Immunology,3600
4,Allergy/Immunology,416


In [128]:
temp2016.npi['count']['Allergy/ Immunology'] += 416

c:\python\python35\lib\site-packages\IPython\core\interactiveshell.py:2862: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [177]:
temp2017 = df2017.groupby('specialty_description', as_index = False).agg({'npi':['count']})

In [127]:
['Allergy/ Immunology']

4026

In [192]:
def find_job(row):
    job = row.specialty_description.item()
    if job in temp2017.specialty_description.values:
        return temp2017[temp2017.specialty_description == job].npi['count'].item()
    return 0

In [195]:
temp2016['count_2017'] = temp2016.apply(find_job, axis = 1)

In [202]:
temp2016['difference'] = temp2016.npi['count'] - temp2016['count_2017']

In [203]:
temp2016['fraction'] = temp2016['difference']/temp2016.npi['count']

In [208]:
max_fraction = temp2016[temp2016.fraction < 1.0].fraction.max()
print(max_fraction)

0.6


In [209]:
temp2016[temp2016.fraction == max_fraction]

,specialty_description,npi,count_2017,difference,fraction
,,count,,,
179,Radiologic Technologist,5,2,3,0.6


# STATS

In [31]:
def gen_seq(N):
    out = []
    for _ in range(1000000):
        arr = np.arange(1, N+1)
        np.random.shuffle(arr)
        temp = arr[0]
        for j in range(len(arr)-1):
            temp += abs(arr[j] - arr[j+1])
        out.append(temp)
    return out

In [32]:
out_10 = gen_seq(10)

In [33]:
print('Mean of N = 10: {}'.format(np.mean(out_10)))

Mean of N = 10: 38.501675


In [34]:
print('STD of N = 10: {}'.format(np.std(out_10)))

STD of N = 10: 6.357895814998465


In [35]:
out_20 = gen_seq(20)

In [36]:
print('Mean of N = 20: {}'.format(np.mean(out_20)))
print('STD of N = 20: {}'.format(np.std(out_20)))

Mean of N = 20: 143.491227
STD of N = 20: 18.440293518121425


In [37]:
print('Prob greater than or equal 45 for N = 10: {}'.format(np.mean([i >= 45 for i in out_10])))

Prob greater than or equal 45 for N = 10: 0.18163


In [38]:
print('Prob greater than or equal 160 for N = 20: {}'.format(np.mean([i >= 160 for i in out_20])))

Prob greater than or equal 160 for N = 20: 0.197842


In [28]:
out_5 = gen_seq(5)

In [30]:
np.mean(out_5)

11.0112